In [1]:
import duckdb
con = duckdb.connect(database=':memory:', read_only=False)



In [11]:
df = con.execute("SELECT [1, 2, 3] as test").fetchdf()
type(df.iloc[0,0])

list

In [12]:
import pandas as pd
import numpy as np
from numpy.random import randint, shuffle
import random

def get_multiple_dfs(data_frame_rows):
    """
    Get a complete code str that creates a DF with random value
    """
    sizes_before_join = int(data_frame_rows * 1.1)
    start_with_offset = int(data_frame_rows * 0.1)
    end_with_offset = start_with_offset + sizes_before_join
    assert sizes_before_join - start_with_offset == data_frame_rows

    id_a = np.arange(sizes_before_join)
    shuffle(id_a)
    a = randint(0,100,size=(sizes_before_join))
    b = randint(0,100,size=(sizes_before_join))
    categories = ['cat_a', 'cat_b', 'cat_c']
    group_col_1 = pd.Series(random.choices(categories, k=sizes_before_join))
    group_col_2 = pd.Series(random.choices(categories, k=sizes_before_join))
    group_col_3 = pd.Series(random.choices(categories, k=sizes_before_join))

    id_b = np.arange(start_with_offset, end_with_offset)
    shuffle(id_b)
    c = randint(0,100,size=(sizes_before_join)) 
    d = randint(0,100,size=(sizes_before_join))

    df_a = pd.DataFrame(zip(id_a, a, b, group_col_1, group_col_2, group_col_3), columns=['id', 'A', 'B', 
        'group_col_1', 'group_col_2', 'group_col_3'])
    df_b = pd.DataFrame(zip(id_b, c, d), columns=['id', 'C', 'D'])
        
    return df_a, df_b

In [13]:
df_a, df_b = get_multiple_dfs(3000000)
con.register('df_a', df_a)
con.register('df_b', df_b)

In [19]:
df_a['lineage_test'] = range(0, len(df_a))
con.register('df_a', df_a)

In [20]:
%%time
df = con.execute("SELECT lineage_test FROM df_a").fetchdf()

CPU times: user 26.2 ms, sys: 14.8 ms, total: 40.9 ms
Wall time: 39.7 ms


In [23]:
%%time
df = con.execute("SELECT [lineage_test] AS lineage_test FROM df_a").fetchdf()

CPU times: user 2.31 s, sys: 133 ms, total: 2.44 s
Wall time: 2.45 s


In [24]:
df_a['lineage_test'] = df['lineage_test']
con.register('df_a', df_a)

In [25]:
%%time
df = con.execute("SELECT lineage_test FROM df_a").fetchdf()

CPU times: user 1.17 s, sys: 172 ms, total: 1.35 s
Wall time: 1.34 s
